In [62]:
import torch
import time
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Load the Models

In [45]:
max_length = 512

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

blen_tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
blen_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

inst_tokenizer = AutoTokenizer.from_pretrained("prakharz/DIAL-BART0")
inst_model = AutoModelForSeq2SeqLM.from_pretrained("prakharz/DIAL-BART0")

# Load the trained checkpoints

In [46]:
model.load_state_dict(torch.load('./model/guideliner.pt'))
blen_model.load_state_dict(torch.load('./model/blenderbot.pt'))
inst_model.load_state_dict(torch.load('./model/intructdialogue.pt'))

model.eval()
blen_model.eval()
inst_model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

# Generate from the Models

When generating the guideline, make sure you follow the below format:
```
<Conversation history where A is the human and B is the robot> | <topic prefences> 
```
*Notice that they are separated by the '|' character*

In [47]:
# Set the input string
in_str = "A: Oh, I've read \"1984\". It\'s a classic. Love it. Have you ever read anything by J.K. Rowling? \
    B: Yes, I love the Harry Potter series. Have you read them? A: Yes, I'm a big fan of Harry Potter. \
        I have read all the books and seen all the movies. Do you have a favorite book in the series?|\
            {\"high-level\": {\"topic\": \"literature\", \"if_interest\": \"yes\"}}, \
                {\"middle-level\": {\"topic\": \"book recommendation\", \"if_interest\": \"unknow\"}}, \
                    {\"low-level\": {\"topic\": \"J.K. Rowling\", \"if_interest\": \"unknow\"}} \
                        {\"high-level\": {\"topic\": \"literature\", \"if_interest\": \"yes\"}}, \
                            {\"middle-level\": {\"topic\": \"book recommendation\", \"if_interest\": \"yes\"}, \"low-level\": {\"topic\": \"Harry Potter\", \"if_interest\": \"yes\"}}"

# Generate the guideline based upon the history and the topic preferences
in_ids = tokenizer(in_str, max_length=max_length, padding='max_length', return_tensors='pt').input_ids
example = model.generate(in_ids, max_new_tokens=50)
guideline = tokenizer.decode(example[0], skip_special_tokens=True)
print("The generated guideline is: ", guideline)

The generated guideline is:  The user is a big fan of Harry Potter and has read all five books in the series. Ask them about their favorite book and suggest similar books they might enjoy.


### BlenderBot Generation

When generating using the Blenderbot model, make sure to use the following format:
```
*ROBOT TEXT*</s> <s>*USER RESPONSE* [GUIDELINE] *GUIDELINE*
or
*ROBOT TEXT*</s> <s>*USER RESPONSE*</s> <s>*ROBOT TEXT*</s> <s>*USER RESPONSE [GUIDELINE] *GUIDELINE*
```
*Notice that each conversation turn by robot and user is separated by "/s" and "s" within <>*

### InstructDialogue Generation

When generating using the Instructdialogue model, make sure to use the following format:
```
Instruction: *INSTRUCTION TEXT*

Input: [TOPICS] *Text talking about the topic of conversation* [CONTEXT] *CONVERSATION HISTORY* [ENDOFDIALOGUE] [QUESTION] *QUESTION TEXT* [GUIDELINE] *GUIDELINE TEXT*

```

```python
# There is a special format for the conversation history that must be followed. Below is an example
"Robot: Hi there, how are you doing today? Human: I'm doing pretty well. [ENDOFTURN] Robot: Same here. I love your earrings, where did you get them? Human: Thanks! I got them from a local boutique."

```

In [65]:
# modify in_str for each model
blend_modified_in = in_str.split('|')[0]
blend_modified_in = blend_modified_in[3:] # remove 'A:'
blend_modified_in = blend_modified_in.replace('B: ', '</s> <s>')
blend_modified_in = blend_modified_in.replace('A: ', '</s> <s>')

#! NEED TO FIGURE OUT HOW TO INSERT CONVERSATION TURNS
#inst_modified_in = in_str.split('|')[0]
#inst_modified_in = inst_modified_in.replace('A:', 'Human:')
#inst_modified_in = inst_modified_in.replace('B:', 'Robot:')
inst_modified_in = "Human: Oh, I've read \"1984\". It\'s a classic. Love it. Have you ever read anything by J.K. Rowling? \
    Robot: Yes, I love the Harry Potter series. Have you read them? Human: Yes, I'm a big fan of Harry Potter. \
    I have read all the books and seen all the movies. Do you have a favorite book in the series?"

topics = "These two people are talking about books."
question = "Given this conversation provided, a response following the topic guideline is"

# Set the input strings
blend_in_str = blend_modified_in + " [GUIDELINE] " + guideline
inst_in_str = "Input: [TOPICS] " + topics + " [CONTEXT] " + inst_modified_in + " [ENDOFDIALOGUE] [QUESTION] " + question + " [GUIDELINE] " + guideline


# Generate the response using the blenderbot model
blend_start = time.time()
blend_in_ids = blen_tokenizer([blend_in_str], max_length=128, return_tensors='pt', truncation=True)
blend_example = blen_model.generate(**blend_in_ids, max_length=60)
blend_response = blen_tokenizer.batch_decode(blend_example, skip_special_tokens=True)[0]
blend_time = (time.time() - blend_start)%60
print("{} sec - Blenderbot generated the following response: {}".format(blend_time, blend_response))

# Generate the response using the instructdialogue model
inst_start = time.time()
inst_in_ids = inst_tokenizer(inst_in_str, max_length=128, return_tensors='pt', truncation=True)
inst_example = inst_model.generate(**inst_in_ids, max_length=60)
inst_response = inst_tokenizer.batch_decode(inst_example, skip_special_tokens=True)[0]
inst_time = (time.time() - inst_start)%60
print("{} sec - Instructdialogue generated the following response: {}".format(inst_time, inst_response))



5.28441858291626 sec - Blenderbot generated the following response:  I think my favorite book is Order of the Phoenix. What's yours?
15.568578481674194 sec - Instructdialogue generated the following response:  Robot: I've read "1984" and "Harry Potter and the Order of the Phoenix", but I haven't seen the movies yet. What's your favorite book in the series?
